In [1]:
%load_ext autoreload
%autoreload 2
import sys,os
sys.path.append('../')   
import numpy as np
import pandas as pd
from model.erroneousPreference import erroneousPreference
from kernel import BlockRBF
from utility import  paramz
from sklearn.model_selection import train_test_split
# for plotting
import matplotlib.pyplot as plt
import arviz as az


#object preferences, but we need to get rid of age, education, sex. Could we  group equal individuals to whom
#were given the same choice set?
df = pd.read_stata("datasets/IoT_DCE_Thermostats-PlosOne.dta")
df

,subject,condition,age,education,stproducttested,choice,label,price,function,item,alternative,choicetask,secbehav,Female,Male
0,2,Graded_A,30,5,Camera,0,0,207.990005,0,1,1,1,4.125,1.0,0.0
1,2,Graded_A,30,5,Camera,1,1,159.990005,0,1,2,1,4.125,1.0,0.0
2,2,Graded_A,30,5,Camera,0,0,223.990005,1,1,3,1,4.125,1.0,0.0
3,2,Graded_A,30,5,Camera,0,0,191.990005,1,2,1,2,4.125,1.0,0.0
4,2,Graded_A,30,5,Camera,1,1,207.990005,1,2,2,2,4.125,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33118,2917,info_label_low,26,6,Wearable,1,1,159.990005,0,9,2,11040,2.750,1.0,0.0
33119,2917,info_label_low,26,6,Wearable,0,0,191.990005,0,9,3,11040,2.750,1.0,0.0
33120,2917,info_label_low,26,6,Wearable,0,0,159.990005,1,10,1,11041,2.750,1.0,0.0
33121,2917,info_label_low,26,6,Wearable,1,1,175.990005,0,10,2,11041,2.750,1.0,0.0


In [3]:
#label preference, Preference 2>1,2>3,2>4, issues a lot of data
df = pd.read_csv("datasets/DDModeChoice.txt", sep="\t", index_col=0)
np.unique(df.values,axis=0).shape
df

,Choice,Distance,School_location,Grade,Age,Gender,CarAvail,Season,CB_location,Leistung
ID,,,,,,,,,,
1,2,6.03139,0,9,15,0,0,0,0,152.98
2,2,3.48652,0,9,15,1,0,0,1,64.67
3,4,5.33100,0,9,14,1,1,0,0,147.19
4,3,3.75005,0,9,15,0,0,0,0,1287.90
5,2,2.04100,0,9,15,1,0,0,1,35.36
...,...,...,...,...,...,...,...,...,...,...
9296,3,8.82052,0,12,19,1,0,1,1,1612.53
9297,3,3.08554,0,12,19,1,0,1,0,140.04
9298,3,6.93700,0,12,18,0,1,1,0,52.42


In [5]:
#object preferences, but we need to get rid of college 	hsg2 	coml5.
#Could we  group equal individuals to whom
#were given the same choice set?
df = pd.read_csv("datasets/Car.csv",index_col=0)
df[(df.college==0) & (df.hsg2==0) & (df.coml5==1)]
df.shape

(300, 40)

In [14]:
df.shape

(300, 40)

In [12]:
np.unique(df.iloc[:,4:].values)

TypeError: '<' not supported between instances of 'int' and 'str'

In [19]:
df.columns

Index(['choice', 'college', 'hsg2', 'coml5', 'type1', 'type2', 'type3',
       'type4', 'type5', 'type6', 'fuel1', 'fuel2', 'fuel3', 'fuel4', 'fuel5',
       'fuel6', 'price1', 'price2', 'price3', 'price4', 'price5', 'price6',
       'speed1', 'speed2', 'speed3', 'speed4', 'speed5', 'speed6',
       'pollution1', 'pollution2', 'pollution3', 'pollution4', 'pollution5',
       'pollution6', 'size1', 'size2', 'size3', 'size4', 'size5', 'size6'],
      dtype='object')

In [4]:
df[(df.college==0) & (df.hsg2==0) ]

,choice,college,hsg2,coml5,type1,type2,type3,type4,type5,type6,...,pollution3,pollution4,pollution5,pollution6,size1,size2,size3,size4,size5,size6
1,choice1,0,0,0,van,regcar,van,stwagon,van,truck,...,0.25,0.25,0.50,0.50,3,3,2,2,3,3
4,choice5,0,0,1,regcar,truck,regcar,van,regcar,stwagon,...,0.25,0.25,0.60,0.60,2,2,3,3,3,3
6,choice5,0,0,0,truck,regcar,truck,van,truck,stwagon,...,0.25,0.25,0.50,0.50,3,3,2,2,3,3
9,choice5,0,0,0,sportuv,sportcar,sportuv,regcar,sportuv,truck,...,0.75,0.75,0.25,0.25,3,3,3,3,2,2
14,choice5,0,0,0,regcar,stwagon,regcar,truck,regcar,van,...,0.40,0.40,0.75,0.75,2,2,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,choice5,0,0,1,van,regcar,van,stwagon,van,truck,...,0.10,0.10,0.40,0.40,3,3,2,2,3,3
285,choice3,0,0,1,regcar,truck,regcar,van,regcar,stwagon,...,0.10,0.10,0.40,0.40,3,3,1,1,2,2
287,choice3,0,0,1,truck,regcar,truck,van,truck,stwagon,...,0.10,0.10,0.40,0.40,3,3,2,2,3,3
289,choice1,0,0,0,truck,regcar,truck,van,truck,stwagon,...,0.40,0.40,0.75,0.75,2,2,3,3,3,3


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


#object preferences, but we need to get rid of age, education, sex. Could we  group equal individuals to whom
#were given the same choice set?
df = pd.read_excel("datasets/Choice experiment data (2).xls",index_col = 0)
#df.dropna(inplace=True)
np.unique(df[['Cost','Energy','Safety']].values,axis=0).shape

[autoreload of numpy.core.overrides failed: Traceback (most recent call last):
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/numpy/core/overrides.py", line 8, in <module>


[autoreload of numpy.lib.npyio failed: Traceback (most recent call last):
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/benavoli/anaconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/npyio.py", line 1381, in <module>
    _lo

TypeError: int() argument must be a string, a bytes-like object or a number, not '_NoValueType'

In [28]:
np.unique(df[pd.isna(df.Cost)].Choice)

array([0, 1])

In [4]:
df = pd.read_csv("datasets/sportscar_choice_long.csv")
df.iloc[0:35,:]

,resp_id,ques,alt,segment,seat,trans,convert,price,choice
0,1,1,1,basic,2,manual,yes,35,0
1,1,1,2,basic,5,auto,no,40,0
2,1,1,3,basic,5,auto,no,30,1
3,1,2,1,basic,5,manual,no,35,0
4,1,2,2,basic,2,manual,no,30,1
5,1,2,3,basic,4,auto,no,35,0
6,1,3,1,basic,5,auto,yes,35,1
7,1,3,2,basic,4,auto,yes,30,0
8,1,3,3,basic,4,manual,no,40,0
9,1,4,1,basic,2,manual,yes,30,0


In [10]:
df = pd.read_csv("datasets/ModeCanada.csv",index_col=0)
df=df[df.noalt==3)]
df

,case,alt,choice,dist,cost,ivt,ovt,freq,income,urban,noalt
35,18,train,0,204,43.40,117,74,4,35,0,3
36,18,bus,0,204,19.87,149,89,8,35,0,3
37,18,car,1,204,38.76,137,0,0,35,0,3
38,19,train,0,282,50.65,174,99,4,45,0,3
39,19,air,0,282,164.20,56,149,9,45,0,3
...,...,...,...,...,...,...,...,...,...,...,...
15480,4313,bus,0,41,18.36,115,95,8,15,0,3
15481,4313,car,1,41,7.79,40,0,0,15,0,3
15514,4322,train,0,347,60.60,193,155,3,35,0,3
15515,4322,air,0,347,155.30,46,120,16,35,0,3


In [3]:
df = pd.read_csv("datasets/train.csv",index_col=0)
df

,id,choiceid,choice,price_A,time_A,change_A,comfort_A,price_B,time_B,change_B,comfort_B
1,1,1,A,2400,150,0,1,4000,150,0,1
2,1,2,A,2400,150,0,1,3200,130,0,1
3,1,3,A,2400,115,0,1,4000,115,0,0
4,1,4,B,4000,130,0,1,3200,150,0,0
5,1,5,B,2400,150,0,1,3200,150,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2925,235,12,A,2550,55,2,1,2550,65,0,1
2926,235,13,B,2550,55,2,1,2100,65,2,0
2927,235,14,A,2100,75,0,1,3000,55,0,0
2928,235,15,B,2100,75,0,1,2100,65,2,0


In [4]:
df.columns

Index(['id', 'choiceid', 'choice', 'price_A', 'time_A', 'change_A',
       'comfort_A', 'price_B', 'time_B', 'change_B', 'comfort_B'],
      dtype='object')

In [5]:
np.unique(df[['price_A', 'time_A', 'change_A',
       'comfort_A', 'price_B', 'time_B', 'change_B', 'comfort_B']],axis=0).shape


(2920, 8)

In [6]:
df[['price_A', 'time_A', 'change_A',
       'comfort_A', 'price_B', 'time_B', 'change_B', 'comfort_B']].shape

(2929, 8)